要求：
    数据切分方式-三七分（测试集30%，训练集70%），随机种子设置为2018
1）数据类型分析
2）无关特征删除
3）数据类型转换
4）缺失值处理

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('data.csv',encoding = 'gbk')
data.head()

,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [3]:
# 划分数据：
label = data.status
data = data.drop(['status'],axis = 1)

In [4]:
# 数据预处理：查看数据情况（数据类型分析）
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4754 entries, 0 to 4753
Data columns (total 89 columns):
Unnamed: 0                                    4754 non-null int64
custid                                        4754 non-null int64
trade_no                                      4754 non-null object
bank_card_no                                  4754 non-null object
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobility

数据一共有4754条，89个变量
数据类型：float(70),int(12),object(7)
数据缺失：int变量无数据缺失，其他都存在大量的数据缺失
缺失大量特征的样本对后续训练用处不大，删除缺失33个特征以上的样本和重复样本

In [5]:
# 删除缺失33个特征以上的样本和重复样本
data_del = data.dropna(thresh=50)
data_del.drop_duplicates(inplace=True)

# 将数据区分为数值型和非数值型
object_column = ['trade_no', 'bank_card_no', 'reg_preference_for_trad', 'source',
                 'id_name', 'latest_query_time', 'loans_latest_time']
data_obj = data_del[object_column]
data_num = data_del.drop(object_column, axis=1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


无关特征删除

In [6]:
# 查看非数值型特征
data_obj.describe()

,trade_no,bank_card_no,reg_preference_for_trad,source,id_name,latest_query_time,loans_latest_time
count,4476,4476,4474,4476,4476,4450,4457
unique,4476,1,5,1,4307,207,232
top,20180504183516981000000381890305,卡号1,一线城市,xs,张龙,2018-04-14,2018-05-03
freq,1,4476,3196,4476,5,423,134


In [7]:
# trade_no是每个交易唯一的交易号，id_name是用户名，这里我认为是无用特征
# 而bank_card_no和source特征的unique值均为1，即所有样本在这两个特征上的值都是一致的，对后续的训练没有帮助。
# 综上，删除bank_card_no、source、trade_no、id_name这四个非数值型特征。
data_obj.drop(['bank_card_no', 'source', 'trade_no', 'id_name'], axis=1, inplace=True)

# 对于非数值型特征，由于数据集并未给出每个特征的含义
# 所以暂时只手动删除custid、Unnamed: 0和有大量缺失值的student_feature这两个特征。
# 其余特征会通过后续数据处理进一步筛选
data_num.drop(['custid', 'student_feature', 'Unnamed: 0'], axis=1, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


缺失值处理：
1）删除 - 删除行（样本）、删除列（特征）
2）填充 - 均值、众数、中位数填充、前值填充

数值型：均值填充
非数值型：前值填充

In [8]:
from sklearn.preprocessing import LabelBinarizer
from sklearn import impute

imputer =  impute.SimpleImputer(strategy='mean')
num = imputer.fit_transform(data_num)
data_num = pd.DataFrame(num, columns=data_num.columns)

data_obj.ffill(inplace=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [9]:
# 非数值型数据的类型转换：对于reg_preference_for_trad，先将其转化为数值型特征，onehot编码
encoder = LabelBinarizer()
reg_preference_1hot = encoder.fit_transform(data_obj['reg_preference_for_trad'])
data_obj.drop(['reg_preference_for_trad'], axis=1, inplace=True)

reg_preference_df = pd.DataFrame(reg_preference_1hot, columns=encoder.classes_)
data_obj = pd.concat([data_obj, reg_preference_df], axis=1)

data_obj.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,latest_query_time,loans_latest_time,一线城市,三线城市,二线城市,其他城市,境外
0,2018-04-25,2018-04-19,1.0,0.0,0.0,0.0,0.0
1,2018-05-03,2018-05-05,1.0,0.0,0.0,0.0,0.0
2,2018-05-05,2018-05-01,1.0,0.0,0.0,0.0,0.0
3,2018-05-05,2018-05-03,0.0,1.0,0.0,0.0,0.0
4,2018-04-15,2018-01-07,1.0,0.0,0.0,0.0,0.0


In [11]:
# 时间数据转化
data_obj['latest_query_time'] = pd.to_datetime(data_obj['latest_query_time'])
data_obj['latest_query_time_month'] = data_obj['latest_query_time'].dt.month
data_obj['latest_query_time_weekday'] = data_obj['latest_query_time'].dt.weekday

data_obj['loans_latest_time'] = pd.to_datetime(data_obj['loans_latest_time'])
data_obj['loans_latest_time_month'] = data_obj['loans_latest_time'].dt.month
data_obj['loans_latest_time_weekday'] = data_obj['loans_latest_time'].dt.weekday

data_obj = data_obj.drop(['latest_query_time', 'loans_latest_time'], axis=1)

data_obj.head()

,一线城市,三线城市,二线城市,其他城市,境外,latest_query_time_month,latest_query_time_weekday,loans_latest_time_month,loans_latest_time_weekday
0,1.0,0.0,0.0,0.0,0.0,4.0,2.0,4.0,3.0
1,1.0,0.0,0.0,0.0,0.0,5.0,3.0,5.0,5.0
2,1.0,0.0,0.0,0.0,0.0,5.0,5.0,5.0,1.0
3,0.0,1.0,0.0,0.0,0.0,5.0,5.0,5.0,3.0
4,1.0,0.0,0.0,0.0,0.0,4.0,6.0,1.0,6.0


合并：将处理后的数值数据和非数值数据进行处理，形成处理后的数据集

In [12]:
data_processed = pd.concat([data_num, data_obj], axis=1)
data_processed.head()

,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,...,loans_latest_day,一线城市,三线城市,二线城市,其他城市,境外,latest_query_time_month,latest_query_time_weekday,loans_latest_time_month,loans_latest_time_weekday
0,0.01,0.99,0.0,0.90,0.55,0.313,17.0,27.0,26.0,3.0,...,18.0,1.0,0.0,0.0,0.0,0.0,4.0,2.0,4.0,3.0
1,0.02,0.94,2000.0,1.28,1.00,0.458,19.0,30.0,14.0,4.0,...,2.0,1.0,0.0,0.0,0.0,0.0,5.0,3.0,5.0,5.0
2,0.04,0.96,0.0,1.00,1.00,0.114,13.0,68.0,22.0,1.0,...,6.0,1.0,0.0,0.0,0.0,0.0,5.0,5.0,5.0,1.0
3,0.00,0.96,2000.0,0.13,0.57,0.777,22.0,14.0,6.0,3.0,...,4.0,0.0,1.0,0.0,0.0,0.0,5.0,5.0,5.0,3.0
4,0.01,0.99,0.0,0.46,1.00,0.175,13.0,66.0,42.0,1.0,...,120.0,1.0,0.0,0.0,0.0,0.0,4.0,6.0,1.0,6.0


In [13]:
# 保存处理后的数据
data_saved = pd.concat([data_processed, label], axis=1)
data_saved.to_csv('data_processed.csv', index=False)

数据标准化处理，数据划分

In [14]:
#数据标准化
from sklearn.preprocessing import StandardScaler
std=StandardScaler()

data_std_x=pd.DataFrame(std.fit_transform(data_processed),columns=data_processed.columns)

# 数据切分
from sklearn.model_selection import train_test_split

X, y = data_std_x.values, label.values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)